# import lib

In [89]:
import numpy as np
import pandas as pd
from google.colab import drive


# Part 1: Load Data

In [90]:
drive.mount('/content/drive')
df = pd.read_csv('PortfolioData.csv', header = None)

df_columns = ['date']
for i in range(12):
  i = str(i)
  df_columns.extend(['unit'+i,'allocated_asset'+i,'return'+i])
df.columns = df_columns

df = df[[c for c in df.columns if 'return' in c]]
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,return0,return1,return2,return3,return4,return5,return6,return7,return8,return9,return10,return11
0,-0.25%,4.79%,-0.80%,0.62%,-3.44%,-1.37%,-4.22%,-1.31%,-0.45%,-1.57%,-1.12%,2.87%
1,-0.24%,7.85%,-2.86%,0.95%,-2.81%,-1.44%,-3.32%,-0.67%,-0.04%,-2.63%,-1.91%,-2.76%
2,-1.04%,3.88%,-2.01%,0.56%,0.18%,-0.74%,0.53%,-0.76%,0.30%,1.26%,1.52%,4.33%
3,-0.75%,-0.18%,-2.25%,1.55%,-1.24%,-1.22%,-1.25%,-0.17%,-0.21%,-1.76%,-0.46%,-5.81%
4,0.52%,-0.83%,0.37%,1.00%,0.85%,0.31%,1.04%,1.06%,0.48%,0.71%,2.15%,-0.84%
...,...,...,...,...,...,...,...,...,...,...,...,...
256,-1.02%,2.25%,-0.73%,-0.11%,-0.15%,-0.84%,0.69%,0.25%,0.79%,3.80%,0.57%,0.76%
257,-2.52%,1.28%,-8.36%,0.25%,-0.66%,-0.71%,-0.57%,-0.96%,-0.20%,-1.59%,-1.56%,0.86%
258,-0.97%,7.19%,-4.61%,-0.24%,-1.29%,-1.83%,-0.83%,1.86%,-1.68%,-4.27%,-3.90%,2.60%
259,0.15%,2.00%,-1.14%,0.24%,-0.15%,-0.34%,0.11%,-0.44%,-2.00%,-5.65%,-2.47%,1.20%


In [91]:
df = df.applymap(lambda x: float(x.replace('%', '')))

## Part 2: Bootstarping Step 1

In [92]:
def bootstrap_samples(returns, B):

  """Constructs B bootstrap samples.
  Args:
    returns: A pandas Series or DataFrame of asset returns.
    B: The number of bootstrap samples.

  Returns:
    A list of B bootstrap samples.
  """

  bootstrap_samples = []
  for _ in range(B):
    bootstrap_sample = np.random.choice(returns.shape[0], size=1, replace=True)
    # print(bootstrap_sample)
    bootstrap_samples.append(returns.iloc[bootstrap_sample])
  # print(bootstrap_samples)
  return np.array(bootstrap_samples)

## Bootstarping Step 2

In [93]:
def bootstrap_statistics(bootstrap_samples):
  """Computes the corresponding mean and covariance matrix for each bootstrap sample.

  Args:
    bootstrap_samples: A list of B bootstrap samples.

  Returns:
    A list of B mean vectors and B covariance matrices.
  """

  means = []
  covariances = []
  for bootstrap_sample in bootstrap_samples:
    # Calculate the mean of the bootstrap sample.
    # display(bootstrap_sample)
    mean = np.mean(bootstrap_sample)
    means.append(mean)

    # Calculate the covariance matrix of the bootstrap sample.
    covariance = np.cov(bootstrap_sample)
    covariances.append(covariance)

  return means, covariances

## Bootstarping Step 3

In [94]:
def calibration(returns, confidence_level,B):

  # Define data sets Cγ1 and Cγ2.
  C1 = []
  C2 = []
  bs_samples = bootstrap_samples(returns, B)
  # print(bs_samples)
  means, covariances = bootstrap_statistics(bs_samples)

  for mean, covariance in zip(means, covariances):
    g1 = (mean - np.mean(returns)).T
    g1 = g1 @ np.linalg.inv(covariance)
    g1 = g1 @ (mean - np.mean(returns))
    g2 = (covariance - np.cov(returns)).trace()
    C1.append(g1)
    C2.append(g2)

  # Calculate the calibrated values of γ1 and γ2.
  gamma1 = np.percentile(C1, 100 - confidence_level)
  gamma2 = np.percentile(C2, 100 - confidence_level)
  print(gamma1)
  return C1,C2

In [95]:
C1,C2 = calibration(df,10,10)
# print(C1)
# print(C2)

[array(22.84510909), array(6.59979697), array(5.15005152), array(2.60740833), array(23.57703864), array(9.80329091), array(7.05407879), array(1.02817197), array(1.29638182), array(4.1446)]


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


LinAlgError: ignored

## Calculate CVaR

In [ ]:
def calculate_cvar(mu, sigma, x, alpha):
  """Calculates the CVaR of a portfolio.

  Args:
    mu: The mean vector of the asset returns.
    sigma: The covariance matrix of the asset returns.
    x: The portfolio weights.
    alpha: The confidence level.

  Returns:
    The CVaR of the portfolio.
  """

  # Calculate the quantile.
  quantile = 1 - alpha

  # Calculate the expected return below the quantile.
  returns = np.dot(mu, x) + np.random.multivariate_normal(np.zeros(len(mu)), sigma)
  below_quantile = returns[returns < quantile]
  cvar = np.mean(below_quantile)

  return cvar


# Part 3: The model

In [ ]:
import numpy as np
from scipy.optimize import minimize

def solve_robust_mean_cvar_without_return_constraint(mu, sigma, alpha):

  """Solves the robust mean-CVaR optimization problem without the expected return constraint.

  Args:
    mu: The mean vector of the asset returns.
    sigma: The covariance matrix of the asset returns.
    alpha: The confidence level.

  Returns:
    A tuple of the global minimum-risk portfolio and the worst-case mean.
  """

  # Initialize the global minimum-risk portfolio and the worst-case mean.
  x_min = np.zeros(len(mu))
  mu_wc = np.inf

  # Iterate over all possible portfolios.
  for x in np.arange(0, 1, 0.01):
    cvar = calculate_cvar(mu, sigma, x, alpha)
    if cvar < mu_wc:
      x_min = x
      mu_wc = cvar

  # Set the minimum return as the worst-case mean.
  return x_min, mu_wc

def solve_robust_mean_cvar(mu, sigma, alpha, rho):
  """Solves the robust mean-CVaR optimization problem with the expected return constraint.

  Args:
    mu: The mean vector of the asset returns.
    sigma: The covariance matrix of the asset returns.
    alpha: The confidence level.
    rho: The target return.

  Returns:
    A portfolio that is robustly efficient for the target return.
  """

  # Constraints.
  constraints = [
      sum(x) == 1,
      rho <= mu.T @ x,
      mu.T @ x >= rho_min,
  ]

  # Objective.
  objective = lambda x: -cvar(mu, sigma, x, alpha)

  # Solve the optimization problem.
  x = optimize.minimize(objective, x0=np.ones(len(mu)), constraints=constraints)

  return x.x

def generate_robust_mean_cvar_efficient_frontier(mu, sigma, alpha, M):
  """Generates the robust Mean-CVaR efficient frontier.

  Args:
    mu: The mean vector of the asset returns.
    sigma: The covariance matrix of the asset returns.
    alpha: The confidence level.
    M: The number of points on the efficient frontier.

  Returns:
    A list of portfolios that are robustly efficient.
  """

  # Initialize the efficient frontier.
  efficient_frontier = []

  # Solve the robust mean-CVaR optimization problem for each target return.
  for rho in np.linspace(rho_min, mu_wc, M):
    x = solve_robust_mean_cvar(mu, sigma, alpha, rho)
    efficient_frontier.append(x)

  return efficient_frontier
